# 🚆Train
지하철 데이터 전처리

### TODO

1. 지하철 정류장 데이터 
   - [X] 데이터 불러오기, 필요없는 로우 및 컬럼 일차 삭제  
   - [ ] 결측값 및 이상값 해결
   - [ ] 서울 및 각 역 위치 지도에 시각화
   - [ ] 수도권 바깥 데이터 삭제 
   - [ ] 필요없는 로우 및 컬럼 삭제 
   - [ ] 환승역, 종점, 분기역 
2. 지하철 시간표 데이터
   - [ ] 데이터 불러오기, 필요없는 로우 및 컬럼 일차 삭제  
   - [ ] 결측값 및 이상값 해결
   - [ ] 방향별 종점별 막차 시간표 정보로 가공

### 0. import

In [65]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from tqdm import trange, notebook

### 1. 지하철 정류장 데이터

##### 1-a. 데이터 불러오기, 필요없는 로우 및 컬럼 일차 삭제  

In [66]:
# 데이터 읽어오기
station = pd.read_csv('train_station.csv')

# 확실히 필요없는 컬럼 제거
station = station.drop(columns=['영문역사명', '한자역사명', '운영기관명', '역사도로명주소', '역사전화번호','데이터기준일자'])

# 확실히 필요없는 행 제거
# 인천 1~2호선
station = station[(station['노선번호'].str.slice(stop=3) != 'S28')]
# 대전
station = station[(station['노선번호'].str.slice(start=1, stop=3) != '30')]
# 대구 1~3호선
station = station[(station['노선번호'].str.slice(start=1, stop=3) != '27')]
# 부산 1~4호선, 김해선
station = station[(station['노선번호'].str.slice(start=1, stop=3) != '26') & (station['노선번호'] != 'L48B1')]
# 광주
station = station[(station['노선번호'].str.slice(start=1, stop=3) != '29')]

# 나중을 위해 역번호 저장
station_id = station['역번호']

station = station.reset_index(drop=True)
station

,역번호,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,역위도,역경도
0,3753,까치울,S1107,도시철도 7호선,일반역,-,-,NaN,NaN
1,3754,부천종합운동장,S1107,도시철도 7호선,일반역,-,-,NaN,NaN
2,3755,춘의,S1107,도시철도 7호선,일반역,-,-,NaN,NaN
3,3756,신중동,S1107,도시철도 7호선,일반역,-,-,NaN,NaN
4,3757,부천시청,S1107,도시철도 7호선,일반역,-,-,NaN,NaN
...,...,...,...,...,...,...,...,...,...
691,4811,달미역,I41WS,서해선,일반역,NaN,NaN,37.349320,126.808218
692,4812,선부역,I41WS,서해선,일반역,NaN,NaN,37.333908,126.810977
693,4813,초지역,I41WS,서해선,환승역,"I4103, I28K1","수인선, 과천안산선",37.320583,126.806151
694,4814,시우역,I41WS,서해선,일반역,NaN,NaN,37.313767,126.798303


##### 1-b. 결측값 및 이상값 확인

In [67]:
# 역이 노선별로 몇 개인지 확인
# 1호선이어도, 구간별로 노선 명칭 다름 (우리가 아는 1호선 = 경원선 + 1호선 + 경인선 + 경부선 + 장항선)
total_station_cnt_by_route = station["노선명"].value_counts().sort_index()
total_station_cnt_by_route

1호선               10
2호선               50
3호선               34
4호선               26
5호선               56
6호선               39
7호선               42
8호선               18
경강선               11
경부선               45
경원선               24
경의중앙선             57
경인선               20
경춘선               24
김포골드라인            10
도시철도 7호선          11
분당선               37
서해선               12
수도권  도시철도 9호선     25
수도권 경량도시철도 신림선    11
수인선               26
신분당선              16
안산과천선             21
에버라인              15
우이신설선             13
의정부               15
인천국제공항선           14
일산선               11
진접선                3
Name: 노선명, dtype: int64

In [68]:
# 역위도, 역경도가 결측값인 역이 노선별로 몇 개인지 확인
null_station_cnt_by_route = station['노선명'][station['역위도'].isnull(
)].value_counts().sort_index()
null_station_cnt_by_route

1호선              10
2호선              50
3호선              34
4호선              26
5호선              56
6호선              39
7호선              42
8호선              18
김포골드라인           10
도시철도 7호선         11
수도권  도시철도 9호선    25
우이신설선            13
의정부              15
진접선               3
Name: 노선명, dtype: int64